In [ ]:
import pandas as pd
import numpy as np

## Creación de datos de ejemplo

In [ ]:
# Crear dataset de pacientes con parámetros hemodinámicos
np.random.seed(42)
n_pacientes = 100

df = pd.DataFrame({
    'paciente_id': range(1, n_pacientes + 1),
    'HR': np.random.randint(40, 140, n_pacientes),  # Heart Rate (latidos/min)
    'SBP': np.random.randint(80, 180, n_pacientes),  # Systolic Blood Pressure (mmHg)
    'SpO2': np.random.randint(85, 100, n_pacientes)  # Saturación de Oxígeno (%)
})

print(df.head(10))

## 1. Función personalizada con apply - Clasificación por fila

In [ ]:
def clasificar_estado_hemodinamico(row):
    """
    Clasifica el estado hemodinámico del paciente según HR, SBP y SpO2
    
    Criterios:
    - Normal: HR 60-100, SBP 90-140, SpO2 >= 95
    - Alerta: 1 parámetro fuera de rango
    - Crítica: 2 o más parámetros fuera de rango
    """
    parametros_anormales = 0
    
    # Evaluar Heart Rate
    if row['HR'] < 60 or row['HR'] > 100:
        parametros_anormales += 1
    
    # Evaluar Systolic Blood Pressure
    if row['SBP'] < 90 or row['SBP'] > 140:
        parametros_anormales += 1
    
    # Evaluar Saturación de Oxígeno
    if row['SpO2'] < 95:
        parametros_anormales += 1
    
    # Clasificar según número de parámetros anormales
    if parametros_anormales == 0:
        return 'Normal'
    elif parametros_anormales == 1:
        return 'Alerta'
    else:
        return 'Crítica'

# Aplicar función por fila
df['estado_apply'] = df.apply(clasificar_estado_hemodinamico, axis=1)

print("\nClasificación usando apply:")
print(df.head(10))
print("\nDistribución de estados:")
print(df['estado_apply'].value_counts())

## 2. Función lambda con apply

In [ ]:
# Clasificación simple usando lambda
df['SpO2_categoria'] = df['SpO2'].apply(
    lambda x: 'Hipoxemia severa' if x < 90 else 
              'Hipoxemia moderada' if x < 95 else 
              'Normal'
)

print("\nCategorización de SpO2 con lambda:")
print(df[['paciente_id', 'SpO2', 'SpO2_categoria']].head(10))

## 3. Operaciones vectorizadas (más eficientes que apply)

In [ ]:
# Clasificación vectorizada - mucho más rápida para grandes volúmenes
def clasificar_vectorizado(df):
    """
    Clasificación usando operaciones vectorizadas de pandas
    Mucho más eficiente que apply para grandes datasets
    """
    # Contar parámetros anormales usando condiciones booleanas
    parametros_anormales = (
        ((df['HR'] < 60) | (df['HR'] > 100)).astype(int) +
        ((df['SBP'] < 90) | (df['SBP'] > 140)).astype(int) +
        (df['SpO2'] < 95).astype(int)
    )
    
    # Clasificar usando np.select
    condiciones = [
        parametros_anormales == 0,
        parametros_anormales == 1,
        parametros_anormales >= 2
    ]
    
    estados = ['Normal', 'Alerta', 'Crítica']
    
    return np.select(condiciones, estados)

df['estado_vectorizado'] = clasificar_vectorizado(df)

print("\nClasificación usando operaciones vectorizadas:")
print(df[['paciente_id', 'HR', 'SBP', 'SpO2', 'estado_vectorizado']].head(10))

## 4. Comparación de rendimiento: apply vs vectorizado

In [ ]:
import time

# Crear dataset más grande para comparar
df_grande = pd.DataFrame({
    'HR': np.random.randint(40, 140, 10000),
    'SBP': np.random.randint(80, 180, 10000),
    'SpO2': np.random.randint(85, 100, 10000)
})

# Medir tiempo con apply
start = time.time()
df_grande['estado_apply'] = df_grande.apply(clasificar_estado_hemodinamico, axis=1)
tiempo_apply = time.time() - start

# Medir tiempo con vectorizado
start = time.time()
df_grande['estado_vectorizado'] = clasificar_vectorizado(df_grande)
tiempo_vectorizado = time.time() - start

print(f"\n=== Comparación de rendimiento (10,000 registros) ===")
print(f"Tiempo con apply: {tiempo_apply:.4f} segundos")
print(f"Tiempo vectorizado: {tiempo_vectorizado:.4f} segundos")
print(f"Mejora de velocidad: {tiempo_apply/tiempo_vectorizado:.2f}x más rápido")

## 5. Verificar que ambos métodos dan el mismo resultado

In [ ]:
# Verificar que los resultados son idénticos
son_iguales = (df['estado_apply'] == df['estado_vectorizado']).all()
print(f"\n¿Los resultados de apply y vectorizado son idénticos? {son_iguales}")

# Mostrar distribución final
print("\nDistribución de estados (método vectorizado):")
print(df['estado_vectorizado'].value_counts())
print(f"\nPorcentajes:")
print(df['estado_vectorizado'].value_counts(normalize=True) * 100)

## 6. Ejemplo adicional: apply por columna

In [ ]:
# Aplicar función a cada columna (axis=0)
print("\n=== Estadísticas por parámetro usando apply por columna ===")

def rango_clasificacion(serie):
    """Clasificar valores en bajo, medio, alto según cuartiles"""
    q1 = serie.quantile(0.25)
    q3 = serie.quantile(0.75)
    return f"Bajo < {q1:.1f} | Medio: {q1:.1f}-{q3:.1f} | Alto > {q3:.1f}"

rangos = df[['HR', 'SBP', 'SpO2']].apply(rango_clasificacion, axis=0)
print(rangos)

## Resumen y Buenas Prácticas

### Cuándo usar apply:
- Lógica compleja que no se puede vectorizar fácilmente
- Funciones que requieren múltiples columnas con lógica condicional compleja
- Datasets pequeños donde el rendimiento no es crítico

### Cuándo usar operaciones vectorizadas:
- Datasets grandes (>1000 registros)
- Operaciones que se pueden expresar con condiciones booleanas
- Cuando el rendimiento es prioritario
- Operaciones matemáticas simples

### Tip importante:
**Siempre intenta vectorizar primero. Solo usa apply cuando la vectorización no sea posible o muy compleja.**